# Implementing Nerual networking using keras library

In [1]:
from random import seed, random, randrange
from csv import reader
from math import exp
import os 
import keras
import keras.utils
import numpy as np
import pickle
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report,confusion_matrix

from keras.models import Sequential
from keras.layers import Dense

from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam


In [2]:
data_path = 'C:/Users/tianyi/Northeastern University/Machine Learning Final Project - Music Classification - Documents'
myspace_mp3s_path = '%s/myspace_mp3s' % data_path
metadata_path = '%s/metadata.json.gz' % myspace_mp3s_path
genre_map_path = '%s/genre_map.pkl' % myspace_mp3s_path
mfcc_path = '%s/audio_features/mfcc' % data_path

# Shuffle, normalize and get mfcc data

In [3]:
def shuffle_data(x, y):
    '''
    Shuffles x and y data. 
    '''
    idx = np.arange(x.shape[0])
    np.random.shuffle(idx)
    x = x[idx]
    y = y[idx]
    
    return x, y


def normalize_data(x):
    '''
    Normalizes x data. 
    '''
    return (x - x.mean()) / x.std()


def get_mfccs(mfcc_path, genres, lab_idx, max_recs):
    '''
    Open and process mfcc and return as x and y arrays.
    '''
    
    # save all x and y values
    mfccs = []
    y = []
    
    # loop over each genre we are working with
    for genre in genres:
        
        # extract path to all song mfccs in current genre
        genre_path = '%s/%s' % (mfcc_path, genre)

        cnt = 0

        # loop over each mfcc in current genre
        for fn in os.listdir(genre_path): 
            
            # get path to mfcc
            fp = '%s/%s' % (genre_path, fn)
            
            # load mfcc 
            # (20, 19518)
            mfcc = np.load(fp) 
            # (20, )
            # take mean
            mfcc = np.mean(mfcc, axis=1)
            mfccs.append(mfcc)
            
            # append target label to list
            y.append(lab_idx[genre])
            
            # for creating balanced dataset
            cnt += 1
            if cnt == max_recs: 
                break
           
    # normalize data and create arrays
    x = np.array(mfccs)
    x = normalize_data(x)
    y = np.array(y)
    
    return shuffle_data(x, y)


SEED = 20211130

# gather all genres, label them and get mfcc

In [4]:
# genre counts (# samples per genre)
genre_cts = {}
for genre in os.listdir(mfcc_path):
    genre_path = '%s/%s' % (mfcc_path, genre)
    genre_cts[genre] = len(os.listdir(genre_path))
   
# print counts for each genre
for g in sorted(genre_cts, key=genre_cts.get, reverse=True):
    print(g, genre_cts[g])

rock 13158
metal 8782
alternative 8778
rap 5906
dance 5624
pop 4684
jazz 4552
hip_hop 4526
experimental 3686
other 3544
world 2225
electronic 2127
folk 1804
punk 1729
blues 1472
ambient 1299
reggae 1114
goth 722
acoustic 678
country 533
house 512
classical 486
spiritual 369
oldies 248
progressive 221
funk 142
easy_listening 131
spoken_word 130
bluegrass 47
industrial 44
showtunes 38
disco 23


Nerual network using keira with 4 genres 

In [5]:
genres = list(genre_cts.keys())
test_genres = ['rock', 'metal','alternative','rap','dance']



print(test_genres)


lab_idx = {g:i for i,g in enumerate(test_genres)}
print('lab_idx', lab_idx)
min_recs = min([genre_cts[g] for g in test_genres]) 
print('min_recs', min_recs)
x1, y1 = get_mfccs(mfcc_path, test_genres, lab_idx,min_recs)

['rock', 'metal', 'alternative', 'rap', 'dance']
lab_idx {'rock': 0, 'metal': 1, 'alternative': 2, 'rap': 3, 'dance': 4}
min_recs 5624


In [6]:
seed(SEED)

l_rate = 0.01
n_epoch = 100
n_hidden = 128
#convert x to one-hot
x1 = x1.tolist()

# convert y to one-hot
y1 = y1.tolist()

#help function that transfer y
def transfer_y(x):
    tmp_y = []
    modified_y = []
    for i in range(len(genres)):
        tmp_y.append(0)
    modified_y = tmp_y
    modified_y[x] = 1
    
    return modified_y

for i in range(len(y1)): 

    y1[i] = transfer_y(y1[i])


X_train1, X_test1, y_train1, y_test1 = train_test_split(x1,y1, test_size=0.2, random_state=SEED)

In [7]:
# Create the model
model1 = Sequential()
model1.add(Dense(32, activation='relu', input_dim=20))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(8, activation='relu'))
model1.add(Dense(32, activation='sigmoid'))

# Compile the model
model1.compile(loss=binary_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
model1.fit(np.array(X_train1), np.array(y_train1),
          batch_size=200,
          epochs=200,
          verbose=1,
          validation_split=0.2)

# Generate generalization metrics
score = model1.evaluate(X_test1, y_test1, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

Epoch 1/200
90/90 [==============================] - 0s 2ms/step - loss: 0.5216 - accuracy: 0.0000e+00 - val_loss: 0.3053 - val_accuracy: 0.0000e+00
Epoch 2/200
90/90 [==============================] - 0s 742us/step - loss: 0.1729 - accuracy: 0.0058 - val_loss: 0.0918 - val_accuracy: 0.1882
Epoch 3/200
90/90 [==============================] - 0s 888us/step - loss: 0.0811 - accuracy: 0.3111 - val_loss: 0.0763 - val_accuracy: 0.3496
Epoch 4/200
90/90 [==============================] - 0s 742us/step - loss: 0.0740 - accuracy: 0.3652 - val_loss: 0.0726 - val_accuracy: 0.3638
Epoch 5/200
90/90 [==============================] - 0s 764us/step - loss: 0.0709 - accuracy: 0.3929 - val_loss: 0.0706 - val_accuracy: 0.3793
Epoch 6/200
90/90 [==============================] - 0s 742us/step - loss: 0.0688 - accuracy: 0.4129 - val_loss: 0.0686 - val_accuracy: 0.4102
Epoch 7/200
90/90 [==============================] - 0s 742us/step - loss: 0.0673 - accuracy: 0.4309 - val_loss: 0.0676 - val_accuracy: 

Epoch 58/200
90/90 [==============================] - 0s 719us/step - loss: 0.0583 - accuracy: 0.5328 - val_loss: 0.0593 - val_accuracy: 0.5296
Epoch 59/200
90/90 [==============================] - 0s 753us/step - loss: 0.0582 - accuracy: 0.5338 - val_loss: 0.0593 - val_accuracy: 0.5304
Epoch 60/200
90/90 [==============================] - 0s 742us/step - loss: 0.0582 - accuracy: 0.5330 - val_loss: 0.0593 - val_accuracy: 0.5296
Epoch 61/200
90/90 [==============================] - 0s 764us/step - loss: 0.0582 - accuracy: 0.5338 - val_loss: 0.0595 - val_accuracy: 0.5249
Epoch 62/200
90/90 [==============================] - 0s 742us/step - loss: 0.0582 - accuracy: 0.5343 - val_loss: 0.0594 - val_accuracy: 0.5333
Epoch 63/200
90/90 [==============================] - 0s 731us/step - loss: 0.0581 - accuracy: 0.5345 - val_loss: 0.0592 - val_accuracy: 0.5342
Epoch 64/200
90/90 [==============================] - 0s 742us/step - loss: 0.0581 - accuracy: 0.5327 - val_loss: 0.0594 - val_accuracy:

Epoch 115/200
90/90 [==============================] - 0s 820us/step - loss: 0.0572 - accuracy: 0.5414 - val_loss: 0.0587 - val_accuracy: 0.5331
Epoch 116/200
90/90 [==============================] - 0s 730us/step - loss: 0.0572 - accuracy: 0.5424 - val_loss: 0.0588 - val_accuracy: 0.5373
Epoch 117/200
90/90 [==============================] - 0s 753us/step - loss: 0.0571 - accuracy: 0.5448 - val_loss: 0.0587 - val_accuracy: 0.5356
Epoch 118/200
90/90 [==============================] - 0s 753us/step - loss: 0.0571 - accuracy: 0.5431 - val_loss: 0.0588 - val_accuracy: 0.5324
Epoch 119/200
90/90 [==============================] - 0s 775us/step - loss: 0.0571 - accuracy: 0.5451 - val_loss: 0.0587 - val_accuracy: 0.5316
Epoch 120/200
90/90 [==============================] - 0s 764us/step - loss: 0.0572 - accuracy: 0.5426 - val_loss: 0.0588 - val_accuracy: 0.5342
Epoch 121/200
90/90 [==============================] - 0s 742us/step - loss: 0.0571 - accuracy: 0.5433 - val_loss: 0.0589 - val_ac

90/90 [==============================] - 0s 719us/step - loss: 0.0567 - accuracy: 0.5495 - val_loss: 0.0585 - val_accuracy: 0.5364
Epoch 172/200
90/90 [==============================] - 0s 742us/step - loss: 0.0567 - accuracy: 0.5495 - val_loss: 0.0584 - val_accuracy: 0.5371
Epoch 173/200
90/90 [==============================] - 0s 764us/step - loss: 0.0567 - accuracy: 0.5475 - val_loss: 0.0586 - val_accuracy: 0.5298
Epoch 174/200
90/90 [==============================] - 0s 775us/step - loss: 0.0567 - accuracy: 0.5487 - val_loss: 0.0584 - val_accuracy: 0.5349
Epoch 175/200
90/90 [==============================] - 0s 775us/step - loss: 0.0567 - accuracy: 0.5472 - val_loss: 0.0584 - val_accuracy: 0.5369
Epoch 176/200
90/90 [==============================] - 0s 787us/step - loss: 0.0567 - accuracy: 0.5467 - val_loss: 0.0584 - val_accuracy: 0.5369
Epoch 177/200
90/90 [==============================] - 0s 899us/step - loss: 0.0567 - accuracy: 0.5477 - val_loss: 0.0585 - val_accuracy: 0.5358

NN using all genres

In [8]:
genres = list(genre_cts.keys())

all_genres =  list(genre_cts.keys())
print(len(genres))
# all_genres = ['jazz', 'reggae']
print(all_genres)


lab_idx = {g:i for i,g in enumerate(all_genres)}
print('lab_idx', lab_idx)
min_recs = min([genre_cts[g] for g in all_genres]) 
print('min_recs', min_recs)
x, y = get_mfccs(mfcc_path, all_genres, lab_idx,min_recs)

32
['acoustic', 'alternative', 'ambient', 'bluegrass', 'blues', 'classical', 'country', 'dance', 'disco', 'easy_listening', 'electronic', 'experimental', 'folk', 'funk', 'goth', 'hip_hop', 'house', 'industrial', 'jazz', 'metal', 'oldies', 'other', 'pop', 'progressive', 'punk', 'rap', 'reggae', 'rock', 'showtunes', 'spiritual', 'spoken_word', 'world']
lab_idx {'acoustic': 0, 'alternative': 1, 'ambient': 2, 'bluegrass': 3, 'blues': 4, 'classical': 5, 'country': 6, 'dance': 7, 'disco': 8, 'easy_listening': 9, 'electronic': 10, 'experimental': 11, 'folk': 12, 'funk': 13, 'goth': 14, 'hip_hop': 15, 'house': 16, 'industrial': 17, 'jazz': 18, 'metal': 19, 'oldies': 20, 'other': 21, 'pop': 22, 'progressive': 23, 'punk': 24, 'rap': 25, 'reggae': 26, 'rock': 27, 'showtunes': 28, 'spiritual': 29, 'spoken_word': 30, 'world': 31}
min_recs 23


In [9]:
seed(SEED)

l_rate = 0.01
n_epoch = 100
n_hidden = 128
#convert x to one-hot
x = x.tolist()

# convert y to one-hot
y = y.tolist()

#help function that transfer y
def transfer_y(x):
    tmp_y = []
    modified_y = []
    for i in range(len(genres)):
        tmp_y.append(0)
    modified_y = tmp_y
    modified_y[x] = 1
    
    return modified_y

for i in range(len(y)): 

    y[i] = transfer_y(y[i])


X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=SEED)

In [10]:



# Create the model
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=20))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(32, activation='sigmoid'))

# Compile the model
model.compile(loss=binary_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

# Fit data to model
model.fit(np.array(X_train), np.array(y_train),
          batch_size=200,
          epochs=500,
          verbose=1,
          validation_split=0.2)

# Generate generalization metrics
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')


Epoch 1/500
3/3 [==============================] - 0s 36ms/step - loss: 0.7044 - accuracy: 0.0383 - val_loss: 0.7019 - val_accuracy: 0.0169
Epoch 2/500
3/3 [==============================] - 0s 7ms/step - loss: 0.6994 - accuracy: 0.0319 - val_loss: 0.6971 - val_accuracy: 0.0085
Epoch 3/500
3/3 [==============================] - 0s 6ms/step - loss: 0.6946 - accuracy: 0.0277 - val_loss: 0.6926 - val_accuracy: 0.0169
Epoch 4/500
3/3 [==============================] - 0s 7ms/step - loss: 0.6900 - accuracy: 0.0255 - val_loss: 0.6881 - val_accuracy: 0.0085
Epoch 5/500
3/3 [==============================] - 0s 6ms/step - loss: 0.6856 - accuracy: 0.0277 - val_loss: 0.6835 - val_accuracy: 0.0254
Epoch 6/500
3/3 [==============================] - 0s 6ms/step - loss: 0.6810 - accuracy: 0.0255 - val_loss: 0.6786 - val_accuracy: 0.0339
Epoch 7/500
3/3 [==============================] - 0s 7ms/step - loss: 0.6761 - accuracy: 0.0298 - val_loss: 0.6733 - val_accuracy: 0.0424
Epoch 8/500
3/3 [=========

3/3 [==============================] - 0s 7ms/step - loss: 0.1628 - accuracy: 0.0255 - val_loss: 0.1658 - val_accuracy: 0.0339
Epoch 60/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1613 - accuracy: 0.0255 - val_loss: 0.1643 - val_accuracy: 0.0339
Epoch 61/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1599 - accuracy: 0.0255 - val_loss: 0.1630 - val_accuracy: 0.0339
Epoch 62/500
3/3 [==============================] - 0s 6ms/step - loss: 0.1586 - accuracy: 0.0255 - val_loss: 0.1617 - val_accuracy: 0.0339
Epoch 63/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1574 - accuracy: 0.0255 - val_loss: 0.1605 - val_accuracy: 0.0339
Epoch 64/500
3/3 [==============================] - 0s 6ms/step - loss: 0.1563 - accuracy: 0.0255 - val_loss: 0.1595 - val_accuracy: 0.0339
Epoch 65/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1552 - accuracy: 0.0255 - val_loss: 0.1585 - val_accuracy: 0.0339
Epoch 66/500
3/3 [===============

3/3 [==============================] - 0s 7ms/step - loss: 0.1403 - accuracy: 0.0447 - val_loss: 0.1431 - val_accuracy: 0.0339
Epoch 118/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1402 - accuracy: 0.0511 - val_loss: 0.1430 - val_accuracy: 0.0424
Epoch 119/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1402 - accuracy: 0.0553 - val_loss: 0.1429 - val_accuracy: 0.0424
Epoch 120/500
3/3 [==============================] - 0s 8ms/step - loss: 0.1401 - accuracy: 0.0596 - val_loss: 0.1428 - val_accuracy: 0.0339
Epoch 121/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1401 - accuracy: 0.0574 - val_loss: 0.1427 - val_accuracy: 0.0339
Epoch 122/500
3/3 [==============================] - 0s 8ms/step - loss: 0.1400 - accuracy: 0.0574 - val_loss: 0.1426 - val_accuracy: 0.0339
Epoch 123/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1400 - accuracy: 0.0574 - val_loss: 0.1425 - val_accuracy: 0.0339
Epoch 124/500
3/3 [========

3/3 [==============================] - 0s 7ms/step - loss: 0.1379 - accuracy: 0.0660 - val_loss: 0.1406 - val_accuracy: 0.0339
Epoch 176/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1378 - accuracy: 0.0638 - val_loss: 0.1406 - val_accuracy: 0.0339
Epoch 177/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1378 - accuracy: 0.0660 - val_loss: 0.1406 - val_accuracy: 0.0339
Epoch 178/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1378 - accuracy: 0.0660 - val_loss: 0.1406 - val_accuracy: 0.0339
Epoch 179/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1377 - accuracy: 0.0596 - val_loss: 0.1405 - val_accuracy: 0.0339
Epoch 180/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1377 - accuracy: 0.0596 - val_loss: 0.1405 - val_accuracy: 0.0339
Epoch 181/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1376 - accuracy: 0.0596 - val_loss: 0.1405 - val_accuracy: 0.0339
Epoch 182/500
3/3 [========

3/3 [==============================] - 0s 7ms/step - loss: 0.1356 - accuracy: 0.0894 - val_loss: 0.1388 - val_accuracy: 0.0508
Epoch 234/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1356 - accuracy: 0.0915 - val_loss: 0.1387 - val_accuracy: 0.0593
Epoch 235/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1356 - accuracy: 0.0894 - val_loss: 0.1387 - val_accuracy: 0.0593
Epoch 236/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1355 - accuracy: 0.0957 - val_loss: 0.1388 - val_accuracy: 0.0593
Epoch 237/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1355 - accuracy: 0.0957 - val_loss: 0.1387 - val_accuracy: 0.0593
Epoch 238/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1354 - accuracy: 0.0915 - val_loss: 0.1387 - val_accuracy: 0.0508
Epoch 239/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1354 - accuracy: 0.0957 - val_loss: 0.1386 - val_accuracy: 0.0593
Epoch 240/500
3/3 [========

3/3 [==============================] - 0s 7ms/step - loss: 0.1334 - accuracy: 0.0957 - val_loss: 0.1370 - val_accuracy: 0.0593
Epoch 292/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1333 - accuracy: 0.0957 - val_loss: 0.1370 - val_accuracy: 0.0593
Epoch 293/500
3/3 [==============================] - 0s 6ms/step - loss: 0.1333 - accuracy: 0.0957 - val_loss: 0.1369 - val_accuracy: 0.0593
Epoch 294/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1333 - accuracy: 0.0979 - val_loss: 0.1369 - val_accuracy: 0.0593
Epoch 295/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1332 - accuracy: 0.1000 - val_loss: 0.1369 - val_accuracy: 0.0593
Epoch 296/500
3/3 [==============================] - 0s 8ms/step - loss: 0.1332 - accuracy: 0.1000 - val_loss: 0.1369 - val_accuracy: 0.0593
Epoch 297/500
3/3 [==============================] - 0s 8ms/step - loss: 0.1331 - accuracy: 0.1064 - val_loss: 0.1368 - val_accuracy: 0.0678
Epoch 298/500
3/3 [========

3/3 [==============================] - 0s 7ms/step - loss: 0.1307 - accuracy: 0.1128 - val_loss: 0.1372 - val_accuracy: 0.0508
Epoch 350/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1306 - accuracy: 0.1149 - val_loss: 0.1372 - val_accuracy: 0.0593
Epoch 351/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1306 - accuracy: 0.1106 - val_loss: 0.1371 - val_accuracy: 0.0593
Epoch 352/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1305 - accuracy: 0.1106 - val_loss: 0.1371 - val_accuracy: 0.0593
Epoch 353/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1305 - accuracy: 0.1106 - val_loss: 0.1371 - val_accuracy: 0.0593
Epoch 354/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1304 - accuracy: 0.1085 - val_loss: 0.1371 - val_accuracy: 0.0593
Epoch 355/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1304 - accuracy: 0.1128 - val_loss: 0.1372 - val_accuracy: 0.0593
Epoch 356/500
3/3 [========

3/3 [==============================] - 0s 7ms/step - loss: 0.1283 - accuracy: 0.1213 - val_loss: 0.1376 - val_accuracy: 0.0593
Epoch 408/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1283 - accuracy: 0.1170 - val_loss: 0.1378 - val_accuracy: 0.0593
Epoch 409/500
3/3 [==============================] - 0s 6ms/step - loss: 0.1282 - accuracy: 0.1170 - val_loss: 0.1378 - val_accuracy: 0.0678
Epoch 410/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1282 - accuracy: 0.1149 - val_loss: 0.1379 - val_accuracy: 0.0593
Epoch 411/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1282 - accuracy: 0.1170 - val_loss: 0.1378 - val_accuracy: 0.0593
Epoch 412/500
3/3 [==============================] - 0s 6ms/step - loss: 0.1281 - accuracy: 0.1191 - val_loss: 0.1376 - val_accuracy: 0.0678
Epoch 413/500
3/3 [==============================] - 0s 6ms/step - loss: 0.1281 - accuracy: 0.1213 - val_loss: 0.1376 - val_accuracy: 0.0678
Epoch 414/500
3/3 [========

3/3 [==============================] - 0s 7ms/step - loss: 0.1263 - accuracy: 0.1170 - val_loss: 0.1386 - val_accuracy: 0.0847
Epoch 466/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1263 - accuracy: 0.1170 - val_loss: 0.1385 - val_accuracy: 0.0763
Epoch 467/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1262 - accuracy: 0.1234 - val_loss: 0.1384 - val_accuracy: 0.0763
Epoch 468/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1262 - accuracy: 0.1234 - val_loss: 0.1382 - val_accuracy: 0.0847
Epoch 469/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1262 - accuracy: 0.1213 - val_loss: 0.1384 - val_accuracy: 0.0847
Epoch 470/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1261 - accuracy: 0.1213 - val_loss: 0.1385 - val_accuracy: 0.0763
Epoch 471/500
3/3 [==============================] - 0s 7ms/step - loss: 0.1261 - accuracy: 0.1213 - val_loss: 0.1385 - val_accuracy: 0.0932
Epoch 472/500
3/3 [========